# 태스크 6: Langchain 에이전트와 Bedrock 모델 통합

이 노트북에서는 에이전트가 사용할 수 있는 도구를 사용하여 작업 순서를 결정하고, 이를 구현하는 계획 및 실행 에이전트를 사용하는 방법을 알아보겠습니다. 

특정 애플리케이션은 사용자의 질문에 답변하기 위해 언어 모델과 다양한 유틸리티에 대한 적응 가능한 호출 순서를 요구합니다. LangChain 에이전트 인터페이스는 유연하며, 외부 도구와 LLM의 추론을 통합할 수 있습니다. 에이전트는 사용자 입력에 따라 사용할 도구를 선택할 수 있습니다. 에이전트는 여러 도구를 사용할 수 있으며 한 도구의 출력을 다음 도구의 입력으로 활용할 수 있습니다.

## 태스크 6.1: 환경 설정

이 태스크에서는 환경을 설정합니다.

In [ ]:
#create a service client by name using the default session.
import math
import numexpr
import json
import datetime
import sys
import os

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

다음으로, Amazon Bedrock에서 호스팅되는 대화형 AI 모델과 상호 작용할 수 있는 LangChain의 ChatBedrock 클래스 인스턴스를 생성합니다.

In [ ]:
#create an instance of the ChatBedrock
from langchain_aws import ChatBedrock

chat_model=ChatBedrock(
    model_id=model_id , 
    client=bedrock_client)

In [ ]:
#invoke model
chat_model.invoke("what is AWS? Answer in a single senetence")

## 태스크 6.2: Synergizing Reasoning and Acting in Language Models 프레임워크

이 태스크에서 ReAct 프레임워크는 대규모 언어 모델이 외부 도구와 상호 작용하여 보다 더 정확하고 사실에 기반한 응답을 제공하는 추가 정보를 얻을 수 있게 합니다.

대규모 언어 모델은 추론에 대한 설명과 작업별 응답을 교대로 생성할 수 있습니다.

추론 설명을 생성하면 모델이 실행 계획을 추론, 모니터링 및 수정할 수 있고, 예상치 못한 시나리오도 처리할 수도 있습니다. 실행 단계에서 모델은 지식 기반 또는 환경과 같은 외부 소스와 인터페이스를 통해 정보를 획득할 수 있습니다.

In [ ]:
from langchain_core.tools import tool

다음 셀에서는 Langchain 프레임워크 내에서 도구 역할을 하는 `calculator` 함수를 정의합니다. 이 도구는 언어 모델이 Python의 numexpr 라이브러리를 사용해 주어진 식을 평가하여 수학적 계산을 수행할 수 있게 합니다. 이 도구는 식이 유효하지 않은 경우를 처리하도록 설계되었습니다. 이 경우, 이 도구는 계산에 대한 접근 방식 재고하도록 모델에 요청합니다.

In [ ]:
@tool
def get_product_price(query:str):
    "Useful when you need to lookup product price"
    import csv
    prices = {}
    try:
        file=open('sales.csv', 'r')
    except Exception as e:
        return ("Unable to look up the price for " + query)
    reader = csv.DictReader(file)
    for row in reader:
        prices[row['product_id']] = row['price']
    file.close()
    qstr=query.split("\n")[0].strip()
    try:
            return ("Price of product "+qstr+" is "+prices.get(qstr)+"\n")
    except:
            return ("Price for product "+qstr+" is not avilable"+"\n")

다음 셀에서는 Langchain 프레임워크 내에서 도구 역할을 하는 `calculator` 함수를 정의합니다. 이 도구를 사용하면 언어 모델에서 Python의 numexpr 라이브러리를 통해 주어진 표현식을 평가하여 수학적 계산을 수행할 수 있습니다. 표현식이 유효하지 않은 사례가 처리되도록 도구가 설계되었습니다. 해당 사례에서는 계산에 대한 접근 방식 재고를 모델에 요청합니다.

In [ ]:
@tool
def calculator(expression: str) -> str:
    """Use this tool to solve a math problems that involve a single line mathematical expression.
    Use math notation and not words. 
    Examples:
        "5*4" for "5 times 4"
        "5/4" for "5 divided by 4"
    """
    try:
        return str(
            numexpr.evaluate(
            expression.strip(),
            global_dict={},  
            local_dict={} # add math constants, if needed
            )
        )
    except Exception as e:
        return "Rethink your approach to this calculation"

In [ ]:
tools = [get_product_price, calculator]

다음 셀에서는 helper 함수를 실행하여 추적 출력을 파일에 인쇄합니다.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
def output_trace(element:str, trace, node=True):
    global trace_handle
    if trace_enabled:
        print(datetime.datetime.now(),file=trace_handle)
        print(("Node: " if node else "Edge: ")+ element, file=trace_handle)
        if element == "ask_model_to_reason (entry)":
            for single_trace in trace:
                print(single_trace, file=trace_handle)
        else:
            print(trace, file=trace_handle)
        print('----', file=trace_handle)
        
def consolidate_tool_messages(message):
    tool_messages=[]
    for msg in message:
        if isinstance(msg, ToolMessage):
            tool_messages.append(msg)
    return tool_messages

## 태스크 6.3: 에이전트 그래프 구축

이 태스크에서는 외부 도구와 상호 작용할 수 있는 대화형 AI 시스템용 에이전트 그래프를 생성하게 됩니다. 에이전트 그래프는 도구와의 대화 및 상호 작용 흐름이 정의되는 상태 머신입니다.

다음 셀에서는 입력에 따라 상태를 업데이트하는 관련 함수가 있는 노드를 정의합니다. 그래프가 한 노드에서 다음 노드로 전환되는 가장자리를 사용하여 노드를 연결합니다. 조건부 에지를 통합하여 특정 조건에 따라 그래프를 다른 노드로 라우팅합니다. 마지막으로 에이전트 그래프를 컴파일하여 정의된 대로 전환 및 상태 업데이트를 처리하여 실행을 준비합니다.

In [ ]:
from typing import Literal
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

# ToolNode is a prebuilt component that runs the tool and appends the tool result to the messages 
tool_node = ToolNode(tools)

# let the model know the tools it can access
model_with_tools = chat_model.bind_tools(tools)
    
# The following function acts as the conditional edge in the graph.
# The next node could be the tools node or the end of the chain.
def next_step(state: MessagesState) -> Literal["tools", "__end__"]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        output_trace("next_step: Proceed to tools",last_message, node=False)
        return "tools"
    output_trace("next_step: Proceed to end",last_message, node=False)
    return "__end__"

#.The following node function invokes the model that has information about the available tools
def ask_model_to_reason(state: MessagesState):
    messages = state["messages"]
    output_trace("ask_model_to_reason (entry)", consolidate_tool_messages(messages))
    try:
        response = model_with_tools.invoke(messages)
    except Exception as e:
        output_trace("ask_model_to_reason", messages)
        output_trace("ask_model_to_reason", "Exception: "+str(e))
        return {"messages": [messages.append("Unable to invoke the model")]}
    output_trace("ask_model_to_reason (exit)", response)
    return {"messages": [response]}


agent_graph = StateGraph(MessagesState)

# Describe the nodes. 
# The first argument is the unique node name, and the second argument is the 
# function or object that will be called when the node is reached
agent_graph.add_node("agent", ask_model_to_reason)
agent_graph.add_node("tools", tool_node)

# Connect the entry node to the agent for the graph to start running
agent_graph.add_edge("__start__", "agent")

# Once the graph transitions to the tools node, the graph will transition to the agent node
agent_graph.add_edge("tools", "agent")

# The transition out of the agent node is conditional. 
# If the output from ask_model_to_reason function included a call to the tools, call the tool; 
# otherwise end the chain 
agent_graph.add_conditional_edges(
    "agent",
    next_step,
)

# Compile the graph definition so that it can run

react_agent = agent_graph.compile()

다음으로 컴파일된 그래프를 시각화합니다. 점선으로 표시된 것처럼 에이전트 노드에서 조건부로 전환되는 것을 관찰합니다.

In [ ]:
from IPython.display import Image, display

try:
    display(Image(react_agent.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

다음 셀에서는 helper 함수를 실행하여 그래프 출력을 인쇄합니다.

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

다음으로 이전 노트북에서 생성한 sales.csv 파일에서 에이전트에게 제품 가격에 대해 물어보고 싶은 질문을 하나 이상 추가합니다.

In [ ]:
#list of questions
questions=[]
questions.append("How much will it cost to buy 3 units of P002 and 5 units of P003?")
#questions.append("How many units of P010 can I buy with $200?")
#questions.append("Can I buy three units of P003 with $200? If not, how much more should I spend to get three units?")
#questions.append("Prices have gone up by 8%. How many units of P003 could I have purchased before the price increase with $140? How many can I buy after the price increase? Fractional units are not pssoible.")

추론에 포함된 단계를 이해하려면 추적을 활성화합니다. 그러나, 위 목록에서 **한 개의 질문을 제외한 모든 질문을 주석 처리**하여 추적 출력을 관리하기 쉽게 유지합니다. 또는 추적을 비활성화하고 모든 질문을 실행할 수도 있습니다.

In [ ]:
trace_enabled=True

if trace_enabled:
    file_name="trace_"+str(datetime.datetime.now())+".txt"
    trace_handle=open(file_name, 'w')

다음 셀에서 위 목록의 질문으로 에이전트를 호출합니다.

In [ ]:
system_message="Answer the following questions as best you can. Do not make up an answer. Think step by step. Do not perform intermediate math calculations on your own. Use the calculator tool provided for math calculations."

for q in questions:
    inputs = {"messages": [("system",system_message), ("user", q)]}
    config={"recursion_limit": 15}
    print_stream(react_agent.stream(inputs, config, stream_mode="values"))
    print("\n"+"================================ Answer complete ================================="+"\n")

if trace_enabled:
    trace_handle.close()